In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
from gemmavqwhisper import GemmaWhisperForConditionalGeneration, Model
from transformers import AutoFeatureExtractor, AutoTokenizer, AutoProcessor
import librosa
import torch

torch.set_grad_enabled(False)

In [3]:
import torch.distributed as dist
import os

def init_distributed_single():
    if not dist.is_initialized():
        dist.init_process_group(
            backend="nccl" if torch.cuda.is_available() else "gloo",
            init_method="tcp://127.0.0.1:23456",
            rank=0,
            world_size=1
        )

init_distributed_single()

In [4]:
model = Model.from_pretrained(
    'mesolitica/gemma3n-audio-encoder-whisper-decoder', torch_dtype = 'auto').train()
model.model.init_quantize_layer('centroids-v2.npy')

Some weights of Model were not initialized from the model checkpoint at mesolitica/gemma3n-audio-encoder-whisper-decoder and are newly initialized: ['model.codebook.weight', 'model.ema_count', 'model.ema_weight', 'model.post_layer_norm.bias', 'model.post_layer_norm.weight', 'model.post_projection.bias', 'model.post_projection.weight', 'model.total_code_usage']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


loaded codebook weight from centroids-v2.npy


In [5]:
same = (model.proj_out.weight == model.model.decoder.embed_tokens.weight).float().mean().tolist()
assert same >= 0.99, "projection is not tied"

In [6]:
model.model.codebook.weight.max()

tensor(5.6397)

In [7]:
model.model.quantize_restart_interval = 1

In [8]:
_ = model.cuda()

In [9]:
feature_extractor = AutoFeatureExtractor.from_pretrained(
    'mesolitica/gemma3n-audio-encoder-whisper-decoder')
tokenizer = AutoTokenizer.from_pretrained('mesolitica/gemma3n-audio-encoder-whisper-decoder')
processor = AutoProcessor.from_pretrained('openai/whisper-large-v3-turbo')

In [10]:
from streaming import MDSWriter, LocalDataset

dataset = LocalDataset('mosaic-stt')

In [11]:
for i in range(3):
    y, sr = librosa.load(dataset[i]['audio_filename'], sr = feature_extractor.sampling_rate)
    features = feature_extractor([y], return_tensors = 'pt')
    features['input_features'] = features['input_features'].cuda()
    features['input_features_mask'] = features['input_features_mask'].cuda()
    features['attention_mask'] = features.pop('input_features_mask')
    input_ids = tokenizer('<|startoftranscript|>' + dataset[i]['text'] + tokenizer.eos_token, 
        add_special_tokens = False, return_tensors = 'pt')['input_ids'].cuda()
    features['labels'] = input_ids[:,1:]
    features['decoder_input_ids'] = input_ids[:,:-1]
    o = model(**features, use_cache = False)
    print(i, model.model.quantize_loss, model.model.quantize_perplexity, model.model.num_active_codes)
    print(i, model.model.total_code_usage.sum().item())

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


tensor([27024,   341,  3275, 15559, 29834, 11490,  6436, 18571, 29371, 12893,
        26248, 31135, 21771, 23693,  9813, 28774, 11253,  7236, 32728, 20986,
        26864, 20810, 29694, 29440, 26354, 12950,  1350, 27814, 30059, 30489,
        11353, 29320, 16193, 13303,   192, 28362, 13868, 22161, 22161, 16128,
         7744], device='cuda:0')
num_update 0
0 tensor(0.2492, device='cuda:0') 39.636878967285156 40
0 40.0
tensor([  597, 11259, 17979, 27277, 23815, 30235, 30235, 26312, 16543, 21771,
        20026, 22545,  4537,   240, 30159,  6588, 28135, 24756,  7713, 11766,
        21143,  6532, 14495, 31286, 10918], device='cuda:0')
num_update 32743
[VQ] Restarted 32743 dead codes.
1 tensor(0.2525, device='cuda:0') 23.651439666748047 24
1 63.0
tensor([  597, 11259, 17979, 27277, 27277, 30235, 30235, 30235, 30235, 30235,
        30235, 26312, 26312, 30235, 26312, 26312, 16543, 30159, 30159, 28135,
        21771,  7713, 11766, 22161, 10918, 21771, 22545,   240, 24756,   597,
        28135, 